In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_linear_schedule_with_warmup
import torch
from torch.utils.data import DataLoader, Dataset
from torch.cuda.amp import autocast, GradScaler
import torch.nn.functional as F

# Validation function
def evaluate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total_samples = 0
    
    with torch.no_grad():
        for batch, labels in dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            labels = labels.to(device)
            
            with autocast():
                outputs = model(**batch)
                loss = criterion(outputs.logits, labels)
                total_loss += loss.item()
            
            preds = torch.argmax(outputs.logits, dim=1)
            correct += (preds == labels).sum().item()
            total_samples += labels.size(0)
    
    avg_loss = total_loss / len(dataloader)
    accuracy = correct / total_samples
    return avg_loss, accuracy

# Training loop with mixed precision and gradient checkpointing
def train_model(model, dataloader, optimizer, scheduler, criterion, device, num_epochs=5):
    model.to(device)
    for epoch in range(num_epochs):
        model.train()
        for batch, labels in dataloader:
            optimizer.zero_grad()
            batch = {k: v.to(device) for k, v in batch.items()}
            labels = labels.to(device)

            with autocast():
                outputs = model(**batch)
                loss = criterion(outputs.logits, labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            scheduler.step()

        val_loss, val_accuracy = evaluate(model, val_dataloader, criterion, device)
        print(f"Epoch {epoch+1}/{num_epochs}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")

if __name__ == '__main__':
    # Model and tokenizer setup
    tokenizer = AutoTokenizer.from_pretrained('roberta-base')
    model = AutoModelForSequenceClassification.from_pretrained('roberta-base', num_labels=NUM_EMOTIONS)
    model.gradient_checkpointing_enable()  # Enable gradient checkpointing
    
    # Mixed precision training setup
    scaler = GradScaler()
    

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.cuda.amp import autocast, GradScaler
from torch.utils.data import DataLoader, Dataset

# Define the Appraisal MLP model
class AppraisalMLP(nn.Module):
    def __init__(self, input_dim, num_emotions):
        super(AppraisalMLP, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, num_emotions)
        )
    
    def forward(self, x):
        return self.fc(x)

# Training loop with mixed precision and gradient checkpointing
def train_appraisal_model(model, dataloader, optimizer, criterion, device, num_epochs=5):
    model.to(device)
    scaler = GradScaler()

    for epoch in range(num_epochs):
        model.train()
        total_train_loss = 0

        for appraisal_inputs, labels in dataloader:
            optimizer.zero_grad()
            appraisal_inputs = appraisal_inputs.to(device)
            labels = labels.to(device)

            with autocast():  # Mixed precision context
                outputs = model(appraisal_inputs)
                loss = criterion(outputs, labels)
                total_train_loss += loss.item()

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

        avg_train_loss = total_train_loss / len(dataloader)
        val_loss, val_accuracy = evaluate(model, val_dataloader, criterion, device)

        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")

# Example of creating the optimizer and criterion
input_dim = 10  # Change to match your dataset's number of appraisal dimensions
num_emotions = 5  # Change to match the number of emotion classes
model = AppraisalMLP(input_dim=input_dim, num_emotions=num_emotions)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

# Assume `train_dataloader` and `val_dataloader` are already defined
train_appraisal_model(model, train_dataloader, optimizer, criterion, device='cuda', num_epochs=10)


In [ ]:
class CombinedModel(nn.Module):
    def __init__(self, num_emotions, appraisal_input_dim):
        super(CombinedModel, self).__init__()
        self.text_model = AutoModelForSequenceClassification.from_pretrained('roberta-base', num_labels=num_emotions)
        self.text_model.gradient_checkpointing_enable()  # Enable gradient checkpointing for the text model
        self.appraisal_model = AppraisalMLP(input_dim=appraisal_input_dim, num_emotions=num_emotions)
        self.fc_combined = nn.Sequential(
            nn.Linear(num_emotions * 2, 128),
            nn.ReLU(),
            nn.Linear(128, num_emotions)
        )
    
    def forward(self, text_inputs, appraisal_inputs):
        text_outputs = self.text_model(**text_inputs).logits
        appraisal_outputs = self.appraisal_model(appraisal_inputs)
        combined = torch.cat((text_outputs, appraisal_outputs), dim=1)
        return self.fc_combined(combined)

# Combined training with mixed precision
def train_combined_model(model, text_dataloader, appraisal_dataloader, optimizer, scheduler, criterion, device, num_epochs=5):
    model.to(device)
    scaler = GradScaler()
    
    for epoch in range(num_epochs):
        model.train()
        for (text_batch, text_labels), appraisal_inputs in zip(text_dataloader, appraisal_dataloader):
            optimizer.zero_grad()
            text_batch = {k: v.to(device) for k, v in text_batch.items()}
            text_labels = text_labels.to(device)
            appraisal_inputs = appraisal_inputs.to(device)

            with autocast():
                outputs = model(text_batch, appraisal_inputs)
                loss = criterion(outputs, text_labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            scheduler.step()

        val_loss, val_accuracy = evaluate(model, val_dataloader, criterion, device)
        print(f"Epoch {epoch+1}/{num_epochs}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")


In [ ]:
def evaluate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total_samples = 0
    
    with torch.no_grad():
        for batch, labels in dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            labels = labels.to(device)
            
            with autocast():
                outputs = model(**batch)
                loss = criterion(outputs.logits, labels)
                total_loss += loss.item()
            
            preds = torch.argmax(outputs.logits, dim=1)
            correct += (preds == labels).sum().item()
            total_samples += labels.size(0)
    
    avg_loss = total_loss / len(dataloader)
    accuracy = correct / total_samples
    return avg_loss, accuracy